In [1]:
# Set device
import torch
device = "cuda" if torch.cuda.is_available() else "cpu"

In [2]:
# ==============================
# Imports
# ==============================
import torch
import pandas as pd
import random
import numpy as np
from collections import Counter
from transformers import XLMRobertaTokenizer, XLMRobertaForMaskedLM
from openprompt.prompts import ManualTemplate, ManualVerbalizer
from openprompt.data_utils import InputExample
from openprompt.plms import load_plm
from openprompt import PromptForClassification, PromptDataLoader
from torch.optim import AdamW
from sklearn.metrics import classification_report
from collections import defaultdict

# ========================================
# Check CUDA
# ========================================
device = "cuda" #if torch.cuda.is_available() else "cpu"
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU name:", torch.cuda.get_device_name(0))
    print("CUDA version:", torch.version.cuda)
    print("GPU count:", torch.cuda.device_count())

# ========================================
# Seeds for reproducibility
# ========================================
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

# ==============================
# Load POS Train Dataset
# ==============================
df = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\pos\train.csv")  # columns: 'word', 'tag'

# Find unique tags
unique_tags = df['tag'].unique()
print("Unique POS tags:", unique_tags)

# Create label map
label_map = {tag: i for i, tag in enumerate(unique_tags)}
print("Label map:", label_map)

# ==============================
# Prepare Few-Shot Training Dataset: first 16 examples per tag
# ==============================
train_dataset = []
guid = 0

for tag in unique_tags:
    tag_rows = df[df['tag'] == tag].head(16)  # first 16 examples
    for _, row in tag_rows.iterrows():
        example = InputExample(
            guid=str(guid),
            text_a=row['word'],
            label=label_map[row['tag']]
        )
        train_dataset.append(example)
        guid += 1

print("✅ Few-shot dataset created with total examples:", len(train_dataset))

# ==============================
# Inspect first few examples
# ==============================
for ex in train_dataset[:20]:
    print("GUID:", ex.guid, "| Word:", ex.text_a, "| Label:", ex.label)

# ==============================
# Define Classes
# ==============================
classes = list(label_map.keys())
print("Classes:", classes)

# ==============================
# Load Pretrained Model and Tokenizer
# ==============================
# # # Step 1: Use load_plm with 'roberta' to get the correct WrapperClass
# _, _, _, WrapperClass = load_plm("roberta", "roberta-base")  # Just to get the wrapper

# # # Step 2: Manually load XLM-RoBERTa model/tokenizer
# model_name = "xlm-roberta-base"
# tokenizer = XLMRobertaTokenizer.from_pretrained(model_name)
# plm = XLMRobertaForMaskedLM.from_pretrained(model_name)

# # ==============================
# # Load Pretrained Language Model (mBERT)
# # ==============================
plm, tokenizer, model_config, WrapperClass = load_plm("bert", "bert-base-multilingual-cased")


C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


CUDA available: True
GPU name: NVIDIA GeForce RTX 4080 SUPER
CUDA version: 12.1
GPU count: 1
Unique POS tags: ['PN' 'G' 'NN' 'P' 'U' 'VB' 'SM' 'PM' 'PP' 'CC' 'ADJ' 'CA' 'RP' 'SC' 'SE'
 'ADV' 'EXP' 'I' 'NEG' 'TA' 'AP' 'Q' 'PD' 'WALA' 'KP' 'GR' 'REP' 'A' 'KD'
 'AA' 'QW' 'KER' 'OR' 'AKP' 'MUL' 'INT' 'AD' 'FR' 'DATE' 'RD']
Label map: {'PN': 0, 'G': 1, 'NN': 2, 'P': 3, 'U': 4, 'VB': 5, 'SM': 6, 'PM': 7, 'PP': 8, 'CC': 9, 'ADJ': 10, 'CA': 11, 'RP': 12, 'SC': 13, 'SE': 14, 'ADV': 15, 'EXP': 16, 'I': 17, 'NEG': 18, 'TA': 19, 'AP': 20, 'Q': 21, 'PD': 22, 'WALA': 23, 'KP': 24, 'GR': 25, 'REP': 26, 'A': 27, 'KD': 28, 'AA': 29, 'QW': 30, 'KER': 31, 'OR': 32, 'AKP': 33, 'MUL': 34, 'INT': 35, 'AD': 36, 'FR': 37, 'DATE': 38, 'RD': 39}
✅ Few-shot dataset created with total examples: 627
GUID: 0 | Word: ’ | Label: 0
GUID: 1 | Word: ای | Label: 0
GUID: 2 | Word: بلال | Label: 0
GUID: 3 | Word: بھائی | Label: 0
GUID: 4 | Word: ستمبر | Label: 0
GUID: 5 | Word: اڑھائی | Label: 0
GUID: 6 | Word: “ | Label: 

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\huggingface_hub\file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForMaskedLM were not initialized fr

In [3]:
# ==============================
# Define Prompt Template
# ==============================
template = ManualTemplate(
    text='لفظ: {"placeholder":"text_a"} → حصہ کلام: {"mask"}',
    tokenizer=tokenizer,
)

 
# Group words by POS tag
tag_to_words = defaultdict(list)

for _, row in df.iterrows():
    word = str(row["word"]).strip()
    tag = row["tag"]
    if word and word not in tag_to_words[tag]:
        tag_to_words[tag].append(word)

# Build verbalizer map
label_words_map = {}

for tag in classes:
    examples = tag_to_words.get(tag, [])

    # Take first 2 words if available
    sample_words = examples[:2]

    # Always include the tag itself
    label_words_map[tag] = [tag] + sample_words


verbalizer = ManualVerbalizer(
    classes=classes,
    label_words=label_words_map,
    tokenizer=tokenizer
)



print("\n📘 FINAL VERBALIZER CONTENT:\n")

for tag, words in label_words_map.items():
    print(f"{tag} → {words}")

 

# ==============================
# Create Prompt Model
# ==============================
prompt_model = PromptForClassification(
    template=template,
    plm=plm,
    verbalizer=verbalizer
)
# prompt_model = prompt_model.to(device)
# prompt_model.to(device)




📘 FINAL VERBALIZER CONTENT:

PN → ['PN', '’', 'ای']
G → ['G', 'میرے', 'میرا']
NN → ['NN', 'بھائی', 'محنت']
P → ['P', 'کا', 'کی']
U → ['U', 'میل', 'من']
VB → ['VB', 'آیاہے', 'ہیں']
SM → ['SM', '۔', '!']
PM → ['PM', '،', '"']
PP → ['PP', 'ہم', 'آپ']
CC → ['CC', 'اور', 'و']
ADJ → ['ADJ', 'قائل', 'میریئٹ']
CA → ['CA', 'اکیس', 'دو']
RP → ['RP', 'خود', 'اپنےآپ']
SC → ['SC', 'جبکہ', 'کہ']
SE → ['SE', 'سے']
ADV → ['ADV', 'برائے', 'زیادہ']
EXP → ['EXP', '”', '(']
I → ['I', 'تو', 'ہی']
NEG → ['NEG', 'نہیں', 'نہ']
TA → ['TA', 'ہیں', 'ہے']
AP → ['AP', 'وہاں', 'اب']
Q → ['Q', 'کچھ', 'ہر']
PD → ['PD', 'اس', 'ان']
WALA → ['WALA', 'والا', 'والے']
KP → ['KP', 'کس', 'کیا']
GR → ['GR', 'اپنی', 'اپنا']
REP → ['REP', 'جس', 'جن']
A → ['A', 'سی', 'سا']
KD → ['KD', 'کسی', 'کس']
AA → ['AA', 'سکتے', 'گیا']
QW → ['QW', 'کیا', 'کیوں']
KER → ['KER', 'کر']
OR → ['OR', 'دونوں', 'ساتویں']
AKP → ['AKP', 'کہیں', 'کیسے']
MUL → ['MUL', 'شاپنگ', 'افطار']
INT → ['INT', 'واہ', 'القادری']
AD → ['AD', 'ایسی']
FR → ['FR', 'آد

In [4]:
# ==============================
# DataLoader for Training
# ==============================
train_loader = PromptDataLoader(
    dataset=train_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=4,
    shuffle=True  # reproducibility preserved by seed
)




prompt_model.train()
optimizer = AdamW(prompt_model.parameters(), lr=1e-5)

for epoch in range(20):
    total_loss = 0
    for batch in train_loader:
        optimizer.zero_grad()
        logits = prompt_model(batch)
        loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")
# ==============================
# Fine-Tuning the Prompt Model
# ==============================
# prompt_model.train()
# optimizer = AdamW(prompt_model.parameters(), lr=1e-5)

# for epoch in range(20):
#     torch.cuda.empty_cache()
#     total_loss = 0
#     for batch in train_loader:
#         optimizer.zero_grad()
#         # Move batch to device
#         batch = {k: v.to(device) if torch.is_tensor(v) else v for k, v in batch.items()}
#         logits = prompt_model(batch)
#         loss = torch.nn.CrossEntropyLoss()(logits, batch['label'])
#         if torch.isnan(loss):
#             print("NaN detected!")
#             print("Logits:", logits)
#             print("Labels:", batch["label"])
#             break        
#         loss.backward()
#         optimizer.step()
#         total_loss += loss.item()
#     print(f"Epoch {epoch+1} Loss: {total_loss:.4f}")

 
 

tokenizing: 627it [00:00, 2515.70it/s]


Epoch 1 Loss: 248.5424
Epoch 2 Loss: 129.0269
Epoch 3 Loss: 87.9056
Epoch 4 Loss: 60.9828
Epoch 5 Loss: 56.5743
Epoch 6 Loss: 38.4644
Epoch 7 Loss: 37.7544
Epoch 8 Loss: 33.3675
Epoch 9 Loss: 28.4013
Epoch 10 Loss: 27.2644
Epoch 11 Loss: 25.8812
Epoch 12 Loss: 27.5564
Epoch 13 Loss: 24.7527
Epoch 14 Loss: 25.6735
Epoch 15 Loss: 29.6333
Epoch 16 Loss: 24.1659
Epoch 17 Loss: 24.1308
Epoch 18 Loss: 22.8382
Epoch 19 Loss: 22.4680
Epoch 20 Loss: 21.0522


In [5]:

# ==============================
# Prepare Evaluation Dataset
# ==============================
df_eval = pd.read_csv(r"C:\Users\stdFurqan\Desktop\paft\pos\test.csv")  # columns: 'word', 'tag'
eval_dataset = [
    InputExample(guid=str(i), text_a=row['word'], label=label_map[row['tag']])
    for i, row in df_eval.iterrows()
]

# ==============================
# DataLoader for Evaluation
# ==============================
eval_loader = PromptDataLoader(
    dataset=eval_dataset,
    tokenizer=tokenizer,
    template=template,
    tokenizer_wrapper_class=WrapperClass,
    max_seq_length=128,
    batch_size=8,
    shuffle=False
)

# ==============================
# Evaluate Model
# ==============================
prompt_model.eval()
all_preds = []
all_labels = []

with torch.no_grad():
    for batch in eval_loader:
        # Move tensors to device
        # batch = {k: v.to(device) if torch.is_tensor(v) else v for k, v in batch.items()}
        logits = prompt_model(batch)
        preds = torch.argmax(logits, dim=-1)
        all_preds.extend(preds.cpu().tolist())
        all_labels.extend(batch['label'].cpu().tolist())

# # ==============================
# # Classification Report
# # ==============================
# print("\n📊 POS Tagging Classification Report:")
# print(classification_report(all_labels, all_preds, target_names=classes, digits=4))
# # ==============================
# # Classification Report (FIXED)
# # ==============================

# all_label_ids = list(range(len(classes)))  # [0, 1, 2, ..., 39]

# print("\n📊 POS Tagging Classification Report:")
# print(
#     classification_report(
#         all_labels,
#         all_preds,
#         labels=all_label_ids,      # 👈 IMPORTANT FIX
#         target_names=classes,
#         digits=4,
#         zero_division=0
#     )
# )


tokenizing: 86676it [00:37, 2325.96it/s]


In [6]:
# ==============================
# Classification Report
# ==============================
print("\n📊 POS Tagging Classification Report:")
print(classification_report(all_labels, all_preds, target_names=classes, digits=4))
# ==============================
# Classification Report (FIXED)
# ==============================

# all_label_ids = list(range(len(classes)))  # [0, 1, 2, ..., 39]

# print("\n📊 POS Tagging Classification Report:")
# print(
#     classification_report(
#         all_labels,
#         all_preds,
#         labels=all_label_ids,      # 👈 IMPORTANT FIX
#         target_names=classes,
#         digits=4,
#         zero_division=0
#     )
# )



📊 POS Tagging Classification Report:
              precision    recall  f1-score   support

          PN     0.3514    0.2851    0.3148      6065
           G     0.4299    0.9515    0.5923       474
          NN     0.7692    0.3514    0.4824     21792
           P     0.9470    0.9806    0.9635     10256
           U     0.0233    1.0000    0.0456        40
          VB     0.6170    0.3049    0.4081     10060
          SM     0.9991    0.9281    0.9623      3464
          PM     0.9918    0.8820    0.9337      1924
          PP     0.8483    0.4958    0.6258      3316
          CC     0.9328    0.9954    0.9631      1938
         ADJ     0.2753    0.1919    0.2261      5342
          CA     0.4731    0.7584    0.5827      1763
          RP     0.8830    0.9881    0.9326        84
          SC     0.9623    0.6518    0.7771      2504
          SE     0.8546    1.0000    0.9216      1440
         ADV     0.3278    0.7351    0.4534      1480
         EXP     0.2306    0.8274    0.3606

C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
C:\Users\stdFurqan\anaconda3\envs\py310\lib\site-packages\sklearn\metrics\_classification.py:1565: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [ ]:
mbert

📊 POS Tagging Classification Report:
              precision    recall  f1-score   support

          PN     0.3514    0.2851    0.3148      6065
           G     0.4299    0.9515    0.5923       474
          NN     0.7692    0.3514    0.4824     21792
           P     0.9470    0.9806    0.9635     10256
           U     0.0233    1.0000    0.0456        40
          VB     0.6170    0.3049    0.4081     10060
          SM     0.9991    0.9281    0.9623      3464
          PM     0.9918    0.8820    0.9337      1924
          PP     0.8483    0.4958    0.6258      3316
          CC     0.9328    0.9954    0.9631      1938
         ADJ     0.2753    0.1919    0.2261      5342
          CA     0.4731    0.7584    0.5827      1763
          RP     0.8830    0.9881    0.9326        84
          SC     0.9623    0.6518    0.7771      2504
          SE     0.8546    1.0000    0.9216      1440
         ADV     0.3278    0.7351    0.4534      1480
         EXP     0.2306    0.8274    0.3606       197
           I     0.7386    0.9389    0.8268      1800
         NEG     0.8669    1.0000    0.9287      1062
          TA     0.5941    0.6677    0.6288      3181
          AP     0.5463    0.9718    0.6994       710
           Q     0.6192    0.5603    0.5883      1219
          PD     0.3517    0.9948    0.5196      1164
        WALA     0.8605    1.0000    0.9250       253
          KP     0.1239    0.6577    0.2086       111
          GR     0.9954    1.0000    0.9977       437
         REP     0.8158    1.0000    0.8986       589
           A     0.1933    0.9175    0.3193       206
          KD     0.9890    0.8404    0.9086       213
          AA     0.5704    0.6363    0.6016      2571
          QW     0.3120    1.0000    0.4756       219
         KER     0.2104    1.0000    0.3476       211
          OR     0.3401    0.8772    0.4902       171
         AKP     0.2668    0.8949    0.4111       257
         MUL     0.0010    0.2222    0.0020        27
         INT     0.0709    0.7966    0.1302        59
          AD     0.2656    1.0000    0.4198        51
          FR     0.0299    1.0000    0.0581        26
        DATE     0.0000    0.0000    0.0000         0
          RD     0.0000    0.0000    0.0000         0

    accuracy                         0.5764     86676
   macro avg     0.5170    0.7576    0.5483     86676
weighted avg     0.6965    0.5764    0.5978     86676